In [1]:
!pip install tensorflow numpy pillow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Custom Focal Loss for Sparse Labels (Fixed)
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fn(y_true, y_pred):
        # Convert sparse labels to one-hot
        y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=3)  # num_classes=3
        y_true = tf.cast(y_true, tf.float32)

        # Compute cross-entropy loss
        ce_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)

        # Compute focal weight
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)  # Probability of true class
        focal_weight = tf.pow(1 - p_t, gamma) * alpha

        # Compute final focal loss
        return tf.reduce_mean(focal_weight * ce_loss)
    return focal_loss_fn

# Cosine Decay Learning Rate Schedule
def cosine_decay(epoch, initial_lr=0.0001):
    epochs_total = 20
    cosine_decay = 0.5 * (1 + np.cos(np.pi * epoch / epochs_total))
    return float(initial_lr * cosine_decay)

# ======================
# 1. DATA PREPARATION
# ======================

dataset = "/content/drive/MyDrive/Skin Disease Dataset Short"
img_size = (300, 300)
batch_size = 32

train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.95, 1.05],
    channel_shift_range=5.0,
    fill_mode='reflect',
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

val_datagen = ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_ds = train_datagen.flow_from_directory(
    dataset,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset="training",
    shuffle=True,
    seed=42
)

val_ds = val_datagen.flow_from_directory(
    dataset,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset="validation"
)

class_names = list(train_ds.class_indices.keys())
print("Detected classes:", class_names)
num_classes = len(class_names)

# ======================
# 2. DISEASE KNOWLEDGE BASE (Unchanged)
# ======================

DISEASE_DATABASE = {
    "Atopic Dermatitis": {
        "description": "Chronic inflammatory skin condition often associated with allergies",
        "severity": "Moderate",
        "treatments": [
            "Use fragrance-free moisturizers daily",
            "Apply topical corticosteroids during flare-ups",
            "Consider wet wrap therapy for severe cases",
            "Identify and avoid triggers like dust or certain fabrics"
        ],
        "when_to_see_doctor": "If symptoms persist after 2 weeks of home care"
    },
    "Eczema": {
        "description": "Condition causing itchy, inflamed skin patches",
        "severity": "Mild-Moderate",
        "treatments": [
            "Apply hydrocortisone cream (1%) to affected areas",
            "Use antihistamines for itch relief",
            "Moisturize with ceramide-based creams",
            "Avoid scratching to prevent infection"
        ],
        "when_to_see_doctor": "If rash covers large areas or shows signs of infection"
    },
    "Psoriasis pictures Lichen Planus and related diseases": {
        "description": "Autoimmune disorder causing rapid skin cell buildup",
        "severity": "Moderate-Severe",
        "treatments": [
            "Apply topical treatments containing salicylic acid",
            "Use prescribed vitamin D analogs",
            "Consider phototherapy treatment",
            "Manage stress which can trigger flare-ups"
        ],
        "when_to_see_doctor": "For diagnosis and prescription treatments"
    }
}

# ======================
# 3. MODEL ARCHITECTURE
# ======================

def build_model():
    base_model = EfficientNetB3(
        weights='imagenet',
        include_top=False,
        input_shape=(*img_size, 3)
    )

    base_model.trainable = True
    for layer in base_model.layers[:100]:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu', kernel_regularizer=l2(1e-4)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_regularizer=l2(1e-4)),
        BatchNormalization(),
        Dense(num_classes, activation='softmax')
    ])
    return model

model = build_model()

# ======================
# 4. TRAINING CONFIGURATION
# ======================

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=focal_loss(gamma=2.0, alpha=0.25),
    metrics=['accuracy']
)

callbacks = [
    tf.keras.callbacks.LearningRateScheduler(cosine_decay),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    tf.keras.callbacks.TerminateOnNaN()
]

# ======================
# 5. MODEL TRAINING (20 EPOCHS)
# ======================

print("\n=== Training ===")
history = model.fit(
    train_ds,
    epochs=20,
    validation_data=val_ds,
    callbacks=callbacks,
    verbose=1
)

# ======================
# 6. PREDICTION & REPORT GENERATION
# ======================

def generate_diagnostic_report(image_path):
    img = Image.open(image_path).convert("RGB")
    img_array = np.array(img.resize(img_size))
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array, verbose=0)
    pred_class = class_names[np.argmax(pred)]
    confidence = np.max(pred)
    disease_info = DISEASE_DATABASE[pred_class]

    plt.figure(figsize=(12, 8))
    plt.subplot(2, 2, 1)
    plt.imshow(img)
    plt.title("Input Image")
    plt.axis('off')

    plt.subplot(2, 2, 2)
    plt.text(0.1, 0.5,
             f"Diagnosis: {pred_class}\n\n"
             f"Confidence: {confidence:.1%}\n\n"
             f"Severity: {disease_info['severity']}\n\n"
             f"Description: {disease_info['description']}",
             fontsize=12)
    plt.axis('off')

    plt.subplot(2, 1, 2)
    treatment_text = "Recommended Treatments:\n\n" + "\n".join(
        f"• {treatment}" for treatment in disease_info['treatments']
    )
    plt.text(0.1, 0.1, treatment_text, fontsize=12)
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    print("\n" + "="*80)
    print(f"🩺 CLINICAL NOTES: {disease_info['when_to_see_doctor']}")
    print("="*80)

    return {
        "diagnosis": pred_class,
        "confidence": float(confidence),
        "severity": disease_info["severity"],
        "treatments": disease_info["treatments"],
        "clinical_notes": disease_info["when_to_see_doctor"]
    }

# Example usage:
# report = generate_diagnostic_report("path_to_image.jpg")

Found 3992 images belonging to 3 classes.
Found 997 images belonging to 3 classes.
Detected classes: ['Atopic Dermatitis', 'Eczema', 'Psoriasis pictures Lichen Planus and related diseases']

=== Training ===
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 310s 2s/step - accuracy: 0.4079 - loss: 0.2881 - val_accuracy: 0.3872 - val_loss: 0.1993 - learning_rate: 1.0000e-04
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - accuracy: 0.5315 - loss: 0.2154 - val_accuracy: 0.4233 - val_loss: 0.2446 - learning_rate: 9.9384e-05
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.5955 - loss: 0.1865 - val_accuracy: 0.4213 - val_loss: 0.3210 - learning_rate: 9.6952e-05
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.6314 - loss: 0.1709 - val_accuracy: 0.4383 - val_loss: 0.3780 - learning_rate: 9.1669e-05
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - accuracy: 0.6656 - loss: 0.1549 - val_accuracy: 0.4524 - val_loss: 0.3891 - learning_rate: 4.1458e-05
Epoch 6

In [19]:
model.save("/content/drive/MyDrive/skin_disease_model.keras")


In [13]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


In [14]:
from pyngrok import ngrok

# Ngrok ka authentication token replace karein apne khud ke token se
!ngrok authtoken 2urmEk8GpIZrzYrnbQ6HQ1oMTJm_447EETWLWdGf8eDgYjckY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [23]:
%%writefile SkinDiseaseDetector.py

import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load the trained model with custom objects
MODEL_PATH = "/content/drive/MyDrive/skin_disease_model.keras"
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fn(y_true, y_pred):
        y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=3)
        y_true = tf.cast(y_true, tf.float32)
        ce_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        focal_weight = tf.pow(1 - p_t, gamma) * alpha
        return tf.reduce_mean(focal_weight * ce_loss)
    return focal_loss_fn

model = tf.keras.models.load_model(MODEL_PATH, custom_objects={"focal_loss_fn": focal_loss()})

# Define class names (Make sure it matches training classes)
class_names = ["Atopic Dermatitis", "Eczema", "Psoriasis Lichen Planus"]

# Disease Knowledge Base
DISEASE_DATABASE = {
    "Atopic Dermatitis": {
        "description": "Chronic inflammatory skin condition often associated with allergies",
        "severity": "Moderate",
        "treatments": [
            "Use fragrance-free moisturizers daily",
            "Apply topical corticosteroids during flare-ups",
            "Consider wet wrap therapy for severe cases",
            "Avoid triggers like dust or certain fabrics"
        ],
        "when_to_see_doctor": "If symptoms persist after 2 weeks of home care"
    },
    "Eczema": {
        "description": "Condition causing itchy, inflamed skin patches",
        "severity": "Mild-Moderate",
        "treatments": [
            "Apply hydrocortisone cream (1%) to affected areas",
            "Use antihistamines for itch relief",
            "Moisturize with ceramide-based creams",
            "Avoid scratching to prevent infection"
        ],
        "when_to_see_doctor": "If rash covers large areas or shows signs of infection"
    },
    "Psoriasis Lichen Planus": {
        "description": "Autoimmune disorder causing rapid skin cell buildup",
        "severity": "Moderate-Severe",
        "treatments": [
            "Apply topical treatments containing salicylic acid",
            "Use prescribed vitamin D analogs",
            "Consider phototherapy treatment",
            "Manage stress which can trigger flare-ups"
        ],
        "when_to_see_doctor": "For diagnosis and prescription treatments"
    }
}

# Streamlit UI
def main():
    st.set_page_config(page_title="Usman - Skin Disease Detector", layout="wide")
    st.title("🩺 Usman - Skin Disease Detector")
    st.markdown("### Upload an image of the affected skin area to detect the disease and get treatment recommendations.")

    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file).convert("RGB")
        st.image(image, caption="Uploaded Image", use_column_width=True)

        # Preprocess Image
        img_size = (300, 300)
        img_array = np.array(image.resize(img_size))
        img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        pred = model.predict(img_array)
        pred_class = class_names[np.argmax(pred)]
        confidence = np.max(pred)
        disease_info = DISEASE_DATABASE.get(pred_class, None)

        st.markdown("---")
        st.subheader(f"💡 Predicted Disease: {pred_class}")
        st.write(f"**Confidence Level:** {confidence:.2%}")

        if disease_info:
            st.write(f"**Description:** {disease_info['description']}")
            st.write(f"**Severity:** {disease_info['severity']}")
            st.markdown("### 🩹 Recommended Treatments:")
            for treatment in disease_info["treatments"]:
                st.write(f"- {treatment}")

            st.markdown("### 🏥 When to See a Doctor:")
            st.warning(disease_info["when_to_see_doctor"])
        else:
            st.error("No information found for this disease.")

        st.markdown("---")
        st.success("✅ This AI-powered detector is designed to assist in preliminary diagnosis. Always consult a healthcare professional for medical advice.")

if __name__ == "__main__":
    main()

Writing SkinDiseaseDetector.py


In [16]:
!ls

drive  sample_data  SkinDiseaseDetector.py


In [17]:
!curl https://loca.lt/mytunnelpassword

34.16.130.102

In [ ]:
!streamlit run SkinDiseaseDetector.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.130.102:8501

⠼⠴⠦your url is: https://tough-months-tan.loca.lt
2025-03-26 19:05:04.918540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743015904.942688   63888 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743015904.949822   63888 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-26 19:05:09.646314: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value 